# Calculating accessible contact volumes

In [1]:
import numpy as np
from matplotlib import pyplot as plt
import pandas as pd
import mdtraj as md
import fretraj as ft
import copy

First, we load a **PDB file** using MDtraj. Here, we look at a preQ1 riboswitch (PDB: 5Q50)

In [2]:
struct = md.load_pdb('preQ1_5q50.pdb')

In [3]:
table, bonds = struct.top.to_dataframe()

In [4]:
table.head()

,serial,name,element,resSeq,resName,chainID,segmentID
0,1,C5',C,1,C,0,
1,2,O5',O,1,C,0,
2,3,C4',C,1,C,0,
3,4,O4',O,1,C,0,
4,5,C3',C,1,C,0,


Next we need to define the **parameters** for the accessible contact volumes. We select `O5'` (serial id: 2) and `O3'` (serial id: 682) as the attachement points on the RNA. FRETraj takes a dictionary based parameter file in json format. The structure is as follows:

In [5]:
labels = {"Position":
            {"Cy3":
                {"attach_id": 2,
                 "linker_length": 20,
                 "linker_width": 5,
                 "dye_radius1": 8.0,
                 "dye_radius2": 3.5,
                 "dye_radius3": 3.0},
            "Cy5":
                {"attach_id": 682,
                 "linker_length": 20,
                 "linker_width": 5,
                 "dye_radius1": 10.0,
                 "dye_radius2": 3.5,
                 "dye_radius3": 3.0}},
          "Distance": {"Cy3-Cy5":
            {"R0": 54}}}

In [6]:
labels = ft.cloud.check_labels(labels, verbose=False)

The parameter file can be saved to disk if desired

In [8]:
ft.cloud.save_labels('out/labels_preQ1.json', labels)

We can now calculate a **donor and acceptor accessible volume** given the parameters defined in `labels`

In [7]:
don = ft.cloud.Volume(struct, 'Cy3', labels)
acc = ft.cloud.Volume(struct, 'Cy5', labels)

We can check the attachment position and the coordinates of the mean position of the dye in the AV

In [8]:
print(f'donor attachment site: {don.resi_atom}')
print(f'acceptor attachment site: {acc.resi_atom}')

print(f'donor MP coordinates: x = {don.acv.mp[0]:.1f}, y = {don.acv.mp[1]:.1f}, z = {don.acv.mp[2]:.1f}')
print(f'acceptor MP coordinates: x = {acc.acv.mp[0]:.1f}, y = {acc.acv.mp[1]:.1f}, z = {acc.acv.mp[2]:.1f}')

donor attachment site: C1-O5'
acceptor attachment site: G33-O5'
donor MP coordinates: x = 28.5, y = 55.3, z = 19.9
acceptor MP coordinates: x = 41.2, y = 31.9, z = 2.8


The AV clouds can be save to disk.

In [9]:
don.save_acv('out/donor_AV.pdb', format='pdb')
acc.save_acv('out/acceptor_AV.pdb', format='pdb')

If PyMOL is installed and launched with the `-R` flag (RPC client), we can connect to it from within this Jupyter notebook

In [11]:
cmd = ft.jupyter.connect2pymol()

ConnectionRefusedError: [Errno 111] Connection refused

In [137]:
isosurf_file = ft.isosurf.__file__.replace('/mnt/c/', 'c:\\').replace('/', '\\')
cmd.run(isosurf_file)

In [116]:
cmd.reinitialize()
cmd.load('preQ1_5q50.pdb')
cmd.load('out/donor_AV.pdb')
cmd.load('out/acceptor_AV.pdb')
cmd.spectrum('count', 'grey10 grey90', 'preQ1_5q50')
cmd.do(f"set_acv_style('donor_AV', 'acceptor_AV', 'Cy3', 'Cy5', {labels}, transparency=True)")

Currently every point in the AV is weighted equally. However cyanine dyes are known to interact with RNA. We can account for this surface sticking by including a contact volume close to the RNA surface. These points are then weighted more strongly. Let's define a **contact volume width** equal to `dye_radius2` and assign a weight to each point in the CV such that the **probability** of finding a dye in the CV is 3x higher than in the free volume (i.e. a relative fraction of 0.75 to 0.25) 

In [118]:
labels_withCV = copy.deepcopy(labels)
labels_withCV['Position']['Cy3']['cv_thickness'] = 5
labels_withCV['Position']['Cy5']['cv_thickness'] = 5

labels_withCV['Position']['Cy3']['cv_fraction'] = 0.75
labels_withCV['Position']['Cy5']['cv_fraction'] = 0.75

In [119]:
don_withCV = ft.cloud.Volume(struct, 'Cy3', labels_withCV)
acc_withCV = ft.cloud.Volume(struct, 'Cy5', labels_withCV)

Let's check if the coordinates of the mean dye position have indeed changed

In [120]:
print(f'donor MP coordinates: x = {don_withCV.acv.mp[0]:.1f}, y = {don_withCV.acv.mp[1]:.1f}, z = {don_withCV.acv.mp[2]:.1f}')
print(f'acceptor MP coordinates: x = {acc_withCV.acv.mp[0]:.1f}, y = {acc_withCV.acv.mp[1]:.1f}, z = {acc_withCV.acv.mp[2]:.1f}')

donor MP coordinates: x = 30.9, y = 49.8, z = 22.5
acceptor MP coordinates: x = 40.3, y = 32.4, z = 8.6


The ACV clouds can be save to disk.

In [121]:
don_withCV.save_acv('out/donor_ACV.pdb', format='pdb')
acc_withCV.save_acv('out/acceptor_ACV.pdb', format='pdb')

Visualize the ACV in PyMOL

In [136]:
cmd.reinitialize()
cmd.load('preQ1_5q50.pdb')
cmd.load('out/donor_ACV.pdb')
cmd.load('out/acceptor_ACV.pdb')
cmd.spectrum('count', 'grey10 grey90', 'preQ1_5q50')
cmd.do(f"set_acv_style('donor_ACV', 'acceptor_ACV', 'Cy3', 'Cy5', {labels}, volume_type='AV', transparency=True)")
cmd.do(f"set_acv_style('donor_ACV', 'acceptor_ACV', 'Cy3', 'Cy5', {labels}, volume_type='CV', transparency=False)")

Next, we calculate the FRET efficiency between the donor and acceptor point cloud

In [20]:
fret = ft.cloud.FRET_Trajectory(don, acc, 'Cy3-Cy5', labels)
fret_withCV = ft.cloud.FRET_Trajectory(don_withCV, acc_withCV, 'Cy3-Cy5', labels)

In [21]:
pd.concat((pd.DataFrame(fret.values, index=['no CV']), pd.DataFrame(fret_withCV.values, index=['CV'])))

,R0 (A),<R_DA> (A),sigma_R_DA (A),<E_DA>,<R_DA_E> (A),R_attach (A),R_mp (A)
no CV,54.0,36.2,10.9,0.85,40.3,25.5,31.7
CV,54.0,28.9,10.1,0.94,34.4,25.5,24.1


The FRET result can also be saved to disk 

In [24]:
fret.save_fret('out/fret_preQ1.json')